In [ ]:
### imports

# external modules
import os
import sys
import json
import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

from automasking.tools.automask_file_parsing import automask_to_map
import plotting.plottools as plottools
import tools.iotools as iotools
import tools.dftools as dftools

In [ ]:
# read automasks from json file

automask_data_dir = '../data/automask_data'
automask_data_file = os.path.join(automask_data_dir, f'automask_2024.json')

with open(automask_data_file, 'r') as f:
    automask_info = json.load(f)

In [ ]:
# set path to parquet files holding the actual data

# settings
datadir = '/eos/user/l/llambrec/dialstools-output'
year = '2024'
dataset = 'ZeroBias'
reco = 'PromptReco'
layers = [1, 2, 3, 4]

# find files
files = {}
for layer in layers:
    me = f'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_{layer}'
    pattern = f'{dataset}-Run{year}?-{reco}-v?-DQMIO-{me}.parquet'
    pattern = os.path.join(datadir, pattern)
    f = glob.glob(pattern)
    files[layer] = f

# existence check
missing = []
for val in files.values():
    for f in val:
        if not os.path.exists(f):
            missing.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')

In [ ]:
# set example lumisections and layer

keys = np.random.choice(list(automask_info.keys()), size=30)
runlumis = [key.split('_') for key in keys]

#runlumis = [(379470, 91)]

#runlumis = [['385178', '218'], ['383855', '639'], ['383363', '130'], ['385324', '1642'], ['382505', '23'], ['382164', '153'], ['384239', '584'], ['384935', '586'], ['383487', '1192'], ['379456', '332'], ['384179', '31'], ['385618', '40'], ['384167', '13'], ['380892', '31'], ['381698', '979'], ['386357', '8126'], ['382585', '10'], ['384753', '453'], ['385057', '75'], ['383368', '270']]

layer = 2

# printouts for testing
print('Will run on following lumisections:')
print(runlumis)

In [ ]:
# loop over example lumisections

for runlumi in runlumis:

    # settings
    run = int(runlumi[0])
    lumi = int(runlumi[1])
    subsystem = f"BPix{layer}"
    key = str(run) + '_' + str(lumi)
    
    # get automask
    automask = automask_info[key][subsystem]
    if len(automask)==0:
        print(f'Skipping {key} since automask is empty for this lumisection')
        continue
    
    # load data
    df = iotools.read_lumisections(files[layer], [run], [lumi], columns=['entries'])
    if df is None: continue
    entries = df['entries'].values[0]
    if entries < 1e5:
        print(f'Skipping {key} since number of entries is quite small')
        continue
    df = iotools.read_lumisections(files[layer], [run], [lumi])
    mes, _, _ = dftools.get_mes(df, xbinscolumn='x_bin', ybinscolumn='y_bin',
                                runcolumn='run_number', lumicolumn='ls_number',
                                runs=[run], lumis=[lumi])
    
    # plot data
    title = f'Raw data for run {run}, LS {lumi}'
    figsize = (18, 9) if layer>=3 else (12,6)
    fig, ax = plottools.plot_hist_2d(mes[0], figsize=figsize, title=title, titlesize=15,
                xaxtitle=None, xaxtitlesize=None, yaxtitle=None, yaxtitlesize=None,
                ticklabelsize=12, colorticklabelsize=12, extent=None, aspect=None, caxrange=None,
                docolorbar=True, caxtitle='Number of clusters', caxtitlesize=15, caxtitleoffset=15,
                origin='lower')
    
    # overlay automask
    automask_map = automask_to_map(automask, subsystem=subsystem)
    ids = np.nonzero(automask_map.astype(int))
    for yidx, xidx in zip(ids[0], ids[1]):
        linewidth = 1 if layer>=3 else 2
        patch = mpl.patches.Rectangle((xidx-0.5, yidx-0.5), 1, 1,
                                      edgecolor='red', linewidth=linewidth,
                                      facecolor='none')
        ax.add_patch(patch)
        
    print(automask)
    plt.show()